Sources  
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/ (pascal dataset)
- https://github.com/keshik6/pascal-voc-classification/tree/master (pascal github inspo)
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/devkit_doc_07-Jun-2007.pdf (pascal devkit)
- https://github.com/FabrizioDeSantis/Object-Detection-Manga109/tree/main (manga109 github inspo)
- manga109 dataset \
- manga109 libraries, api 
- manga109 paper 

Dependencies 

In [13]:
import manga109api_custom
from visualize_data import draw_annotations
from IPython.display import display
from sklearn.model_selection import train_test_split

Access Data
- data split does not account for different author styles 

In [14]:
# directory with Manga109 data 
manga109_root_dir = "./Manga109/Manga109_released_2023_12_07" # directory with Manga109 data

# custom parser to access Manga109 data 
p = manga109api_custom.Parser(root_dir=manga109_root_dir)

# images = [{'img_path','page_annotation'},...] # dictionary of images 
# author_labels = ['book1','book2','book1',...] # corresponding book label associated with each image
images, book_labels = p.load_all_images()

# define the proportions for initial split (e.g., 80% train/validation, 20% test) #CHECK 
train_val_ratio = 0.8
test_ratio = 0.2

# split data into training/validation and testing sets
train_val_images, test_images, train_val_labels, _ = train_test_split(
    images, book_labels, stratify=book_labels, test_size=test_ratio, random_state=42
)

# further split the training/validation set into training and validation sets
train_images, val_images, _, _ = train_test_split(
    train_val_images, train_val_labels, stratify=train_val_labels, test_size=1 - train_val_ratio, random_state=42
)

Transformations

Visualize Data 

In [52]:
print(images[0]['img_path'])

C:\Users\Grace\Documents\Manga109-BoundingDetection\Manga109\Manga109_released_2023_12_07\images\ARMS\000.jpg


In [55]:
from PIL import Image
# int = 56
# 
# image = Image.open(images[int]['img_path'])
# display(image)

# butt = draw_annotations(image, images[int]['page_annotation'])
# display(butt)
# 
# int1 = 4356
# image1 = Image.open(images[int1]['img_path'])
# 
# butt1 = draw_annotations(image1, images[int1]['page_annotation'])
# display(butt1)

img = Image.open("C:\Users\Grace\Documents\Manga109-BoundingDetection\Manga109\Manga109_released_2023_12_07\images\Hamlet\000.jpg")
display(img)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2391597484.py, line 16)

- Optimizer
- Training 
- Validate 
- Testing 
- Testing Evalution 